# A Test Mode for `SmartPlug`

## Introduction

Developing the `SmartPlug` class has been fun and I would like to build on it. It's a bit unfortunate that at this point I can't really work on `SmartPlug` when I'm not at home, because I can't test anything I write without access to a FRITZ!Box with the relevant home automation devices. I would like to have a way to be able to test mode when I'm on the road.

For that purpose, I would have to simulate the behavior of the FRITZ! smart plugs. I came up with an idea, but before I can explain it, let's take a brief look at how `SmartPlug` works (code snippet below). 

### Review of the `SmartPlug` class

`SmartPlug` is built on top of the class `HomeAutomationDevice` from the component `fritzconnection.lib.fritzhomeauto` of the `fritzconnection` module. But to be clear: it *does not inherit from* `HomeAutomationDevice` in the ordinary Python sense. Rather, the `SmartPlug` constructor takes an instance of `HomeAutomationDevice`, places it as a private attribute, and the remaining class definition borrows some a few attributes and methods from the `HomeAutomationDevice` instance. Here's a stripped down version of the code:


```python
# A stripped down version of the defition of SmartPlug

from fritzconnection.lib.fritzhomeauto import FritzHomeAutomation

class SmartPlug():

    def __init__(self,fritzdevice,idle_threshold=5):
        # private attribute containing a HomeAutomationDevice() instance
        self.__device = fritzdevice
        # some attributes borrowed from the HomeAutomationDevice() instance
        self.identifier = self.__device.identifier
        self.DeviceName = self.__device.DeviceName
        self.model = f"{self.__device.Manufacturer} {self.__device.ProductName}"
        # additional attribute: threshold for power in idle state (measured in Watts)
        self.idle_threshold = idle_threshold

    # some methods borrowed from the HomeAutomationDevice() instance
    def is_switchable(self):
        return self.__device.is_switchable()
    def get_switch_state(self):
        return self.__device.get_switch_state()
    def set_switch(self,arg):
        return self.__device.set_switch(arg)
    def get_basic_device_stats(self):
        return self.__device.get_basic_device_stats()
    
    # some customized methods
    def get_power_stats(self):
        """Get the power statistics recorded by the smart plug."""
        power_stats = self.__device.get_basic_device_stats()['power']
        return power_stats
    
    ...
```

### Strategy for simulating FRITZ! smart plugs

The important thing to understand is how `SmartPlug` interacts with FRITZ! smart plugs, because that's the only things that's missing without access to a suitable FRITZ! home network. This interaction mainly takes place *through the borrowed* methods from `HomeAutomationDevice`. Currentely, these are four: `is_switchable`, `get_switch_state` `set_switch`, `get_basic_device_stats`.

If I want to simulated this behavior, I only have to mimic these four methods.

The crucial observation is that the entire network interaction of a `SmartPlug` instance passes through the `HomeAutomationDevice` instance in the `__device` attribute. 

Here's the general idea: Instead of passing an actual `HomeAutomationDevice` to the the `fritzdevice` argument of the `SmartPlug` constructor, I can pass anything that *behaves like* such an instance. So let's define that type of thing. 

In the following I will define a class `HomeAutomationDeviceSimulator` that has mimcs all the relevant attributes and methods of `HomeAutomationDevice`. Since I'm currently only interested in smart plugs and their power measurements, I'll keep it simple and ignore all other aspects. However, in the future, I might also want to work with thermostats and radiator controls. This would require further work.

## Building the class `HomeAutomationDeviceSimulator` 

Let's start with the attributes. The current implementation of `SmartPlug` makes use of four `HomeAutomationDevice` attributes: `DeviceName`, `Manufacturer`, `ProductName`, and `identifier`. The latter is not really that important, it holds a 12 digit integer (split into a 5+7 digits) which uniquely identifies the hardware (like a MAC address). 

In [2]:
class HomeAutomationDeviceSimulator:

    def __init__(self, name="Smart Plug Simulator"):
        self.DeviceName = name 
        self.Manufacturer = "SB4D" 
        self.ProductName = "FRITZ! Home Auto Simulator"
        self.identifier = "12345 1234567"

As for the methods, I currently need to mimic the four `HomeAutomationDevice` methods `is_switchable`, `get_switch_state` `set_switch`, `get_basic_device_stats`. For now I'll add placeholders.

In [3]:
class HomeAutomationDeviceSimulator:

    def __init__(self, name="Smart Plug Simulator"):
        # attributes explicitly used by SmartPlug
        self.DeviceName = name 
        self.Manufacturer = "SB4D" 
        self.ProductName = "FRITZ! Home Auto Simulator"
        self.identifier = "12345 1234567"
    
    def is_switchable(self):
        pass

    def get_switch_state(self):
        pass
    
    def set_switch(self,arg):
        pass
    
    def get_basic_device_stats(self):
        pass


While implementing the methods, we will need to add more attributes that mimic corresponding attributes of `HomeAutomationDevice` instances.

### The easy part: simulating power switching

Let's start building the methods. Since I'm currently only interested in smart plugs, I will simply have `is_switchable` return `True`.

In [4]:
class HomeAutomationDeviceSimulator:

    def __init__(self, name="Smart Plug Simulator"):
        # attributes explicitly used by SmartPlug
        self.DeviceName = name 
        self.Manufacturer = "SB4D" 
        self.ProductName = "FRITZ! Smart Plug Simulator"
        self.identifier = "12345 1234567"
    
    def is_switchable(self):
        # Assuming that all devices are smart plugs...
        return True

    def get_switch_state(self):
        pass
    
    def set_switch(self,arg):
        pass
    
    def get_basic_device_stats(self):
        pass


For the mehtods `get_switch_state` and `set_switch` we we add an additional attribute `_switch_state`. As you can see, I've chosen to make it a private attribute. There's no real reason, but I find it helpful to distinguish the *implicitly used* attributes from the *explicitly used* ones.

In [5]:
class HomeAutomationDeviceSimulator:

    def __init__(self, name="Smart Plug Simulator"):
        # attributes explicitly used by SmartPlug
        self.DeviceName = name 
        self.Manufacturer = "SB4D" 
        self.ProductName = "FRITZ! Smart Plug Simulator"
        self.identifier = "12345 1234567"
        # attributes implicitly used by SmartPlug 
        # by way of the methods below
        self._switch_state == True
    
    def is_switchable(self):
        """Checks whether a switchable device is simulated."""
        # Assuming that all devices are smart plugs...
        return True

    def get_switch_state(self):
        """Check whether the simulated device's power switch 
        is on or off."""
        return self.__switch_state
    
    def set_switch(self,target_state:bool):
        """Sets the power switch state of the simulated device."""
        self._switch_state = target_state
    
    def get_basic_device_stats(self):
        pass


### The hard part: simulating power measurements

The real challenge is modeling the real world behavior of the method `get_basic_device_stats`. Here's a first approximation of what happens when you call it in a FRITZ! home automation system:
* Suppose that `homeautodevice` is some instance of `HomeAutomationDevice` that corresponds to some physical home automation device.
* When `hometautohomeautodevice.get_basic_device_stats()` is called, a series of processes in the `fritzconnection` module is triggered, to the effeft that a message is prepared and transmitted the FRITZ!Box router using some protocol whose specifics shall not concern us. 
* The message instructs the FRITZ!Box to forward a request to the phyisical home automation device to return the currently available device statistics, presumably stored in internal memory of the device.
* The device sends this information to the FRITZ!Box which sends it back to our computer and `fritzconnection` parses the received information as a dictionary which is eventually returned by `hometautohomeautodevice.get_basic_device_stats()`.

Here's some code that makes this happen using my `sb4dfritz` package as a short cut.


In [16]:
from sb4dfritz import FritzBoxSession

fritzbox = FritzBoxSession()
home_auto_devices = fritzbox.home_auto.get_homeautomation_devices()
homeautodevice = home_auto_devices[3]


In [ ]:
print("type(homeautodevice) =", type(homeautodevice))
print("type(homeautodevice) =", homeautodevice)
device_stats = homeautodevice.get_basic_device_stats()


type(homeautodevice) = <class 'fritzconnection.lib.fritzhomeauto.HomeAutomationDevice'>
type(homeautodevice) = ain: 08761 0406876, AVM - FRITZ!DECT 200


dict_keys(['temperature', 'voltage', 'power', 'energy'])

In [28]:
device_stats['power']['data']

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


```
device_stats
    device_stats['temperature']
        ...
    device_stats['voltage']
        ...
    device_stats['power']
        device_stats['power']['count'] 
            60 <- integer, count of available recorded measurements
        device_stats['power']['grid']
            10 <- integer, time between measurements in seconds
        device_stats['power']['datatime']
            datetime.datetime(YYYY, MM, DD, HH, MM, SS) 
                <- time stamps of latest measurement with second precision
        device_stats['power']['data']
            [value1, value2, ...] 
                <- list of integers, power measurements
                <- value1 is the latest measurement
                <- a value of 100 indicates 1 Watt
    device_stats['energy']
        ...
```